## Magentic One

![magentic](https://www.microsoft.com/en-us/research/uploads/prod/2024/11/magnetic_example-2048x1155.png)

In [6]:
%pip install markitdown playwright

  Using cached playwright-1.49.1-py3-none-manylinux1_x86_64.whl.metadata (3.5 kB)
  Using cached greenlet-3.1.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (3.8 kB)
  Using cached pyee-12.0.0-py3-none-any.whl.metadata (2.8 kB)
Using cached playwright-1.49.1-py3-none-manylinux1_x86_64.whl (44.2 MB)
Using cached greenlet-3.1.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (602 kB)
Using cached pyee-12.0.0-py3-none-any.whl (14 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### `magentic_one.py` core code

```python
def __init__(self, client: ChatCompletionClient, hil_mode: bool = False):
    self.client = client
    self._validate_client_capabilities(client)

    fs = FileSurfer("FileSurfer", model_client=client)
    ws = MultimodalWebSurfer("WebSurfer", model_client=client)
    coder = MagenticOneCoderAgent("Coder", model_client=client)
    executor = CodeExecutorAgent("Executor", code_executor=LocalCommandLineCodeExecutor())
    agents: List[ChatAgent] = [fs, ws, coder, executor]
    if hil_mode:
        user_proxy = UserProxyAgent("User")
        agents.append(user_proxy)
    super().__init__(agents, model_client=client)

```

In [1]:
# Autonomously complete a coding task:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.teams.magentic_one import MagenticOne
from autogen_agentchat.ui import Console

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o",
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key
)



client = azure_openai_chat_completion_client
m1 = MagenticOne(client=client)
task = "마이크로소프트 현재 주가와 전망에 대해 알려줘"
result = await Console(m1.run_stream(task=task))
print(result)

/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/models/openai/_openai_client.py:412: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["vision"] is False:
/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/models/openai/_openai_client.py:427: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["json_output"] is False and json_output is True:
/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/models/openai/_openai_client.py:433: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["function_calling"] is False and len(tools) > 0:
/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:149: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4

---------- user ----------
마이크로소프트 현재 주가와 전망에 대해 알려줘
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

마이크로소프트 현재 주가와 전망에 대해 알려줘


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.
WebSurfer: 
    A helpful assistant with access to a web browser.
    Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, etc., filling in form fields, etc.).
    It can also summarize the entire page, or answer questions based on the content of the page.
    It can also be asked to sleep and wait for pages to load, in cases where the pages seem to be taking a while to load.
Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.
Executor: A computer terminal that performs no other action than running Python scripts (provided to it quoted in ```python code bl

/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:284: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, json_output=True)


---------- WebSurfer ----------
Web surfing error:

Traceback (most recent call last):
  File "/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py", line 377, in on_messages_stream
    content = await self._generate_reply(cancellation_token=cancellation_token)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py", line 404, in _generate_reply
    await self._lazy_init()
  File "/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py", line 261, in _lazy_init
    browser = await self._playwright.chromium.launch(**launch_args)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-pac

/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:343: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


...

/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:503: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(


---------- WebSurfer ----------
I typed 'Microsoft stock market outlook 2023' into '2000 문자 중 0'.

 Here is a screenshot of the webpage: [Microsoft stock market outlook 2023 - 검색](https://www.bing.com/search?q=Microsoft+stock+market+outlook+2023&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=E10071A01BE74C44AA7BE13474CFD21C&ghsh=0&ghacc=0&ghpl=).
 The viewport shows 20% of the webpage, and is positioned at the top of the page 
The following metadata was extracted from the webpage:

{
    "meta_tags": {
        "referrer": "origin-when-cross-origin",
        "SystemEntropyOriginTrialToken": "A1L3tx5CzccqjN3lK6st/fXMwhf9EeokCPf8XCt0DVI8JPbg37BWq0zKvlqgkdm8YEUbthoGkC/xdR1+iIz4txAAAABxeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiTXNVc2VyQWdlbnRMYXVuY2hOYXZUeXBlIiwiZXhwaXJ5IjoxNzM5NzI0MzExLCJpc1N1YmRvbWFpbiI6dHJ1ZX0=",
        "og:description": "Bing\uc740 \uc9c0\ub2a5\uc801\uc778 \uac80\uc0c9 \uae30\ub2a5\uc740 \uc0ac\uc6a9\uc790\uac00 \uc6d0\ud558\ub294 \uc815\ubcf4\ub97

/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:853: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(messages, cancellation_token=cancellation_token)


---------- MagenticOneOrchestrator ----------
현재 마이크로소프트(Microsoft, MSFT)의 주가는 약 $427.85입니다. 이에 대한 전망으로, 분석가들은 MSFT의 목표 주가를 평균 약 $501.8에서 $509.4로 설정하고 있으며, 이는 현재 주가에서 약 20-21% 상승할 가능성을 나타냅니다. 예측 범위는 최저 $455에서 최고 $600까지 다양합니다.

마이크로소프트의 강력한 시장 위치와 클라우드 인프라 서비스 분야에서의 성장이 주가 상승에 긍정적인 영향을 미칠 것으로 보입니다. 전반적으로, 마이크로소프트의 주식에 대한 시장 전망은 낙관적이며, 지속적인 성장과 수익성을 기대할 수 있을 것 같습니다.
---------- Summary ----------
Number of messages: 25
Finish reason: We have successfully retrieved and provided both the current stock price and the market outlook for Microsoft.
Total prompt tokens: 27037
Total completion tokens: 767
Duration: 296.80 seconds
TaskResult(messages=[TextMessage(source='user', models_usage=None, content='마이크로소프트 현재 주가와 전망에 대해 알려줘', type='TextMessage'), TextMessage(source='MagenticOneOrchestrator', models_usage=None, content="\nWe are working to address the following user request:\n\n마이크로소프트 현재 주가와 전망에 대해 알려줘\n\n\nTo answer this request we have assembled the following team:\n\nFileSurfer: An agent

/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:400: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)
